In [ ]:
# Copyright (C) 2016-2018 by Jakob J. Kolb at Potsdam Institute for Climate
# Impact Research
#
# Contact: kolb@pik-potsdam.de
# License: GNU AGPL Version 3
%matplotlib inline
import PyDSTool as pdt
import numpy as np
import networkx as nx
from random import shuffle
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt
from pydivest.macro_model.integrate_equations_aggregate import IntegrateEquationsAggregate as MacroModel
from pydivest.micro_model.divestmentcore import DivestmentCore as MicroModel
from pymofa.experiment_handling import even_time_series_spacing


In [ ]:
# investment_decisions:

nopinions = [50, 50]

# Parameters:

phi, b_d = 0.5, 3.

input_parameters = {'b_c': 1., 'phi': .0, 'tau': 1.,
                    'eps': 0.01, 'b_d': 4., 'e': 1.,
                    'b_r0': 0.1,
                    'possible_cue_orders': [[0], [1]],
                    'xi': .12, 'd_k': 0.06, 'd_c': 0.1, 'kappa_c': 0.5,
                    'L': 100., 'C': 1., 'G_0': 500000.,
                    'campaign': False, 'learning': True,
                    'interaction': 1, 'test': False,
                    'R_depletion': False}

# investment_decisions
opinions = []
for i, n in enumerate(nopinions):
    opinions.append(np.full((n), i, dtype='I'))
opinions = [item for sublist in opinions for item in sublist]
shuffle(opinions)

# network:
N = sum(nopinions)
p = .2

while True:
    net = nx.erdos_renyi_graph(N, p)
    if len(list(net)) > 1:
        break
adjacency_matrix = nx.adj_matrix(net).toarray()

# investment
clean_investment = np.ones(N)
dirty_investment = np.ones(N)

init_conditions = (adjacency_matrix, opinions,
                   clean_investment, dirty_investment)


In [ ]:
m = MicroModel(*init_conditions, **input_parameters)

t_m = 0
xis = []
t_len = 30
data_points = 21
xi_max = .135
for xi in np.linspace(0.12, xi_max, data_points):
    print(xi)
    t_m += t_len
    xis += [xi]*t_len
    m.xi = xi
    m.run(t_max=t_m)

trj = even_time_series_spacing(m.get_aggregate_trajectory(), t_len*data_points)
trj['xi'] = xis

In [ ]:
xi_means = trj.groupby('xi').mean()
xi_std = trj.groupby('xi').std()

In [ ]:
fig, axes = plt.subplots(ncols=3)
fig.set_figwidth(15)

trj[['C']].plot(ax=axes[0])
trj[['x', 'z']].plot(ax=axes[1])

xi_means[['C']].plot(ax=axes[2], style='bo')
axb = axes[2].twinx()
xi_means[['x']].plot(ax=axb, style='go')

#print(trj.mean(axis=0))

In [ ]:
data_path = "../output_data/X7"
with pd.HDFStore(data_path+'.h5') as store:
    mean = store.select('dat_0')
plt_data = mean[['C', 'z', 'x', 'xi']].reset_index().set_index(['tstep'])
#plt_data.columns = plt_data.columns.droplevel(0)
plt_data[['C']].plot()

In [ ]:
data_path = "../output_data/X7"
with pd.HDFStore(data_path+'/mean_trj.h5') as store:
    mean = store.select('dat_1')
mean[['C', 'x']].reset_index().set_index(['xi'])[['C']].plot()

In [ ]:
fig, axes = plt.subplots(ncols=2)
fig.set_figwidth(10)
axes[0].plot(pts['t'], pts['K_cc'], label='$K_c^c$')
axes[0].plot(pts['t'], pts['K_cd'], label='$K_c^d$')
axes[0].plot(pts['t'], pts['K_dc'], label='$K_d^c$')
axes[0].plot(pts['t'], pts['K_dd'], label='$K_d^d$')
axes[0].plot(pts['t'], pts['C'], label='C')
axes[0].legend()

axes[1].plot(pts['t'], pts['x'], label='x')
axes[1].plot(pts['t'], pts['y'], label='y')
axes[1].plot(pts['t'], pts['z'], label='z')
axes[1].legend()



In [ ]:
ode.set(ics=pts[-1])

In [ ]:
PC = pdt.ContClass(ode)
PCargs = pdt.args(name='EQ1', type='EP-C')
PCargs.freepars = ['xi']
PCargs.MaxNumPoints = 200
PCargs.MaxStepSize = 2
PCargs.MinStepSize = 1e-2
PCargs.StepSize = 1e-1
PCargs.verbosity = 1
PCargs.SaveEigen = True
# PCargs.LocBifPoints = ['LP', 'B']
PCargs.LocBifPoints = 'all'
PCargs.StopAtPoints = ['B']

In [ ]:
PC.newCurve(PCargs)

In [ ]:
import time

%matplotlib inline
for i in range(100):
    start = time.clock()
    PC['EQ1'].forward()
    stop = time.clock()
    print(i, stop-start)
    PC['EQ1'].display(coords=('xi', 'C'), axes=(2, 2, 1), stability=True)
    PC['EQ1'].display(coords=('xi', 'K_dc'), axes=(2, 2, 2), stability=True)
    PC['EQ1'].display(coords=('xi', 'x'), axes=(2, 2, 3), stability=True)
    PC['EQ1'].display(coords=('xi', 'z'), axes=(2, 2, 4), stability=True)
    PC['EQ1'].plot.togglePoint('LP1')
    fig = plt.gcf()
    fig.set_figwidth(8)
    fig.set_figheight(8)
    for ax, title in zip(fig.get_axes(), ['C', r'$K_d^c$', 'x', 'z']):
        ax.set_title('')
        ax.set_xlabel(r'$\gamma$')
        ax.set_ylabel(title)
    fig.tight_layout()
    fig.savefig(f'fig_ba_{i}.png')
    fig.clear()
    

In [ ]:
%matplotlib inline
#fig, axes = plt.subplots(ncols=4)
PC['EQ1'].display(coords=('xi', 'C'), axes=(1, 4, 1), stability=True)
PC['EQ1'].display(coords=('xi', 'K_dc'), axes=(1, 4, 3), stability=True)
PC['EQ1'].display(coords=('xi', 'x'), axes=(1, 4, 2), stability=True)
PC['EQ1'].display(coords=('xi', 'z'), axes=(1, 4, 4), stability=True)
PC['EQ1'].plot.togglePoint('LP1')
fig = plt.gcf()
fig.set_figwidth(12)
fig.set_figheight(3)
for ax, title, tag in zip(fig.get_axes(), ['C', r'$K_d^c$', 'x', 'z'], ['a)', 'b)', 'c)', 'd)']):
    ax.set_title('')
    ax.set_xlabel(r'$\gamma$')
    ax.set_ylabel(title)
    ax.text(x=.04, y=.9, s=tag,
        fontdict={'fontsize': 16},
        transform=ax.transAxes)
fig.tight_layout()

fig.savefig(f'fig_ba_wide.pdf')

In [ ]:
#PC['EQ1'].plot.togglePoints(visible='off', byname=['LP1', 'LP2', 'P2', 'P1'])
PC['EQ1'].plot.toggleLabels(visible='off', byname=['LP3', 'LP4', 'B', 'B1'])
PC['EQ1'].plot.refresh()
PC['EQ1'].plot.fig1.fig.savefig('ba_plot.pdf')
PC['EQ1'].plot.fig1.fig

In [ ]:
2**1.2